In [108]:
from collections import Counter
from enum import IntEnum
from pathlib import Path
import shutil
import subprocess
from tqdm import tqdm
from typing import Dict, List, Optional

In [63]:
IWASM = Path(shutil.which("iwasm"))

PY2WASM = Path(shutil.which("py2wasm"))

PYTHON_COOKBOOK = Path.cwd().joinpath("./cookbook/src")
assert PYTHON_COOKBOOK.exists(), "use `git submodule update` to fetch cookbook"


In [87]:
class CaseResult(IntEnum):
    OK = 0
    COMPILATION_FAILURE = 1
    EXECUTION_FAILURE = 2
    DIFFERENT_RESULT = 3

For each case, we are going to compile the python script to a WebAssembly core module, and compare both execution results of python and WebAssembly.

In [116]:
def compile_py_2_wasm(py2wasm_bin: Path, py_file: Path, out_dir: Optional[Path]) -> Path:
    """
    print both stdout and stderr when running the command and meeting an error
    """
    if not out_dir is None:
        out_dir.mkdir(parents=True, exist_ok=True)
        wasm_file = out_dir.joinpath(py_file.stem + ".wasm")
    else:
        wasm_file = py_file.with_suffix(".wasm")
    
    cmd = f"{py2wasm_bin} {py_file} -o {wasm_file}"
    try:
        subprocess.run(cmd, cwd=Path.cwd(), capture_output=True, check=True, shell=True, universal_newlines=True, text=True)
    except subprocess.CalledProcessError as e:
        print(e.stderr)
        raise e

    return wasm_file

def execute_py(py_file: Path) -> subprocess.CompletedProcess:
    cmd = f"python {py_file}"
    return subprocess.run(cmd, cwd=Path.cwd(), capture_output=True, check=True, shell=True, universal_newlines=True, text=True)

def execute_wasm(iwasm_bin: Path, wasm_file: Path) -> subprocess.CompletedProcess:
    cmd = f"{iwasm_bin} --dir=. {wasm_file}"
    try:
        p = subprocess.run(cmd, cwd=Path.cwd(), capture_output=True, check=True, shell=True, universal_newlines=True, text=True)
    except subprocess.CalledProcessError as e:
        print(e.stdout)
        print(f"{'-' * 50}")
        print(e.stderr)
        raise e

    return p

def compare_result(expected: str, actual: str) -> bool:
    """
    Compare the expected and actual result, print them out if they are different
    """
    if expected != actual:
        print(f"{'<' * 50}")
        print(f"{expected}")
        print(f"{'-' * 50}")
        print(f"{actual}")
        print(f"{'>' * 50}")
        return False

    return True

def execute_case(py_file: Path, out_dir: Optional[Path]) -> CaseResult:
    try:
        wasm_file = compile_py_2_wasm(PY2WASM, py_file, out_dir)
    except subprocess.CalledProcessError:
        return CaseResult.COMPILATION_FAILURE

    try:
        p = execute_wasm(IWASM, wasm_file)
    except subprocess.CalledProcessError as e:
        return CaseResult.EXECUTION_FAILURE

    output_from_wasm = p.stdout

    p = execute_py(py_file)
    output_from_py = p.stdout

    if not compare_result(output_from_py, output_from_wasm):
        return CaseResult.DIFFERENT_RESULT
    
    return CaseResult.OK

def visit_case(py_file: Path, out_dir: Optional[Path]) -> CaseResult:
    print(f"py_file={py_file}, out_dir={out_dir}")
    return CaseResult.OK

def execute_test(chapter: Path, bypass: List[str]) -> Dict[str, CaseResult]:
    # list all sub-directories in chapter
    chapter_tests = [x for x in chapter.iterdir() if x.is_dir()]

    # walk
    ret = {}
    for test_dir in tqdm(chapter_tests):
        if test_dir.name in bypass:
            print(f"🎈 bypass {test_dir}")
            continue

        # assume all .py under the test_dir are test cases
        py_files = [x for x in test_dir.iterdir() if x.suffix == ".py"]

        for py_file in py_files:
            result = execute_case(py_file, None)
            # result = visit_case(py_file, None)
            case_name = f"{test_dir.name}.{py_file.stem}"
            ret[case_name] = result
    
    return ret

In [91]:
%%script echo skipping

test_py_file = PYTHON_COOKBOOK.joinpath("1/calculating_with_dictionaries/example.py")
out_dir =  Path.cwd().joinpath("out")
execute_case(test_py_file, out_dir)


skipping


In [90]:
%%script echo skipping

test_py_file = PYTHON_COOKBOOK.joinpath("1/transforming_and_reducing_data_at_the_same_time/example.py")
out_dir =  Path.cwd().joinpath("out")
execute_case(test_py_file, out_dir)

skipping


Walk through Chapter 1, collect all case files(*.py), compile them to WebAssembly core modules, execute them in both .py and .wasm, and compare the results

In [117]:
chapter_1 = PYTHON_COOKBOOK.joinpath("1")
# bypass temporay list
bypass_tests = [
    "transforming_and_reducing_data_at_the_same_time", 
    "sort_a_list_of_dictionaries_by_a_common_key"
]

test_result = execute_test(chapter_1, bypass_tests)
Counter(test_result.values())

# print failures
for k,v in test_result.items():
    if v != CaseResult.OK:
        print(k)

 75%|███████▌  | 12/16 [03:26<01:06, 16.53s/it]

🎈 bypass /workspaces/wamr-with-py/cookbook/src/1/transforming_and_reducing_data_at_the_same_time


 88%|████████▊ | 14/16 [03:43<00:26, 13.01s/it]


--------------------------------------------------
Traceback (most recent call last):
  File "./example.py", line 12, in <module>
FileNotFoundError: [Errno 44] No such file or directory: 'somefile.txt'



100%|██████████| 16/16 [04:19<00:00, 16.22s/it]

{'calculating_with_dictionaries.example.py': <CaseResult.OK: 0>, 'sort_a_list_of_dictionaries_by_a_common_key.example.py': <CaseResult.OK: 0>, 'removing_duplicates_from_a_sequence_while_maintaining_order.example2.py': <CaseResult.OK: 0>, 'removing_duplicates_from_a_sequence_while_maintaining_order.example.py': <CaseResult.OK: 0>, 'implementing_a_priority_queue.example.py': <CaseResult.OK: 0>, 'grouping-records-together-based-on-a-field.grouping.py': <CaseResult.OK: 0>, 'filtering_list_elements.example.py': <CaseResult.OK: 0>, 'working_with_multiple_mappings_as_a_single_mapping.example.py': <CaseResult.OK: 0>, 'finding_out_what_two_dictionaries_have_in_common.example.py': <CaseResult.OK: 0>, 'finding_the_largest_or_smallest_n_items.example.py': <CaseResult.OK: 0>, 'mapping_names_to_sequence_elements.example1.py': <CaseResult.OK: 0>, 'unpack_a_fixed_number_of_elements_from_iterables_of_arbitrary_length.example.py': <CaseResult.OK: 0>, 'extracting_a_subset_of_a_dictionary.example.py': <Ca

Counter({<CaseResult.OK: 0>: 15, <CaseResult.EXECUTION_FAILURE: 2>: 1})